# 사용된 XGBoost Feature

- Meta Feature (문장 길이, Stop words 갯수, ..., Named Entity)
- FastText Embedding
- Naive Bayes
- Logistic Regression
- SGDClassifier
- RandomForestClassifier
- MLPClassifier
- DecisionTreeClassifier

In [1]:
import pandas as pd
import numpy as np
import re

data_path = '../../kaggle_data/fiction_author/'

train = pd.read_csv(data_path + 'train.csv', encoding='utf-8')
test = pd.read_csv(data_path + 'test_x.csv', encoding='utf-8')

In [2]:
train.head()

,index,text,author
0,0,"He was almost choking. There was so much, so m...",3
1,1,"“Your sister asked for it, I suppose?”",2
2,2,"She was engaged one day as she walked, in per...",1
3,3,"The captain was in the porch, keeping himself ...",4
4,4,"“Have mercy, gentlemen!” odin flung up his han...",3


In [3]:
test.head()

,index,text
0,0,“Not at all. I think she is one of the most ch...
1,1,"""No,"" replied he, with sudden consciousness, ""..."
2,2,As the lady had stated her intention of scream...
3,3,“And then suddenly in the silence I heard a so...
4,4,His conviction remained unchanged. So far as I...


### textstat

- Textstat는 텍스트에서 통계를 계산하는 데 사용하기 쉬운 라이브러리입니다. 가독성, 복잡성 및 등급 수준을 결정하는 데 도움이 됩니다.

### fasttext

- 단어를 벡터로 만드는 또 다른 방법으로는 페이스북에서 개발한 FastText가 있습니다.
- Word2Vec 이후에 나온 것이기 때문에, 메커니즘 자체는 Word2Vec의 확장이라고 볼 수 있습니다. 
- Word2Vec와 FastText와의 가장 큰 차이점이라면 Word2Vec는 단어를 쪼개질 수 없는 단위로 생각한다면, FastText는 하나의 단어 안에도 여러 단어들이 존재하는 것으로 간주합니다. 즉 내부 단어(subword)를 고려하여 학습합니다.

In [4]:
!pip install textstat
!pip install fasttext

## 1) flesch_reading_ease

- Flesch reading-ease test 에서는 점수가 높을수록 읽기 쉬운 재료를 나타내며 숫자가 낮을수록 읽기 어려운 구절을 표시합니다.
- Flesch Reading Ease Score를 반환합니다.
- 최대 점수는 121.22점이지만 점수가 얼마나 낮을 수 있는지에 대한 제한은 없습니다. 음수 점수가 유효합니다.

## 2) nltk

- 교육용으로 개발된 자연어 처리 및 문서 분석용 파이썬 패키지. 다양한 기능 및 예제를 가지고 있으며 실무 및 연구에서도 많이 사용됩니다.
- 말뭉치(corpus)는 자연어 분석 작업을 위해 만든 샘플 문서 집합을 말한다. 단순히 소설, 신문 등의 문서를 모아놓은 것도 있지만 품사. 형태소, 등의 보조적 의미를 추가하고 쉬운 분석을 위해 구조적인 형태로 정리해 놓은 것을 포함한다.
- 말뭉치 자료는 설치시에 제공되지 않고 download 명령으로 사용자가 다운로드 받아야 한다.
- nltk.download("book") 명령을 실행하면 NLTK 패키지 사용자 설명서에서 요구하는 대부분의 말뭉치를 다운로드 받아준다.

### 2-1) nltk.tokenize

- 자연어 문서를 분석하기 위해서는 우선 긴 문자열을 분석을 위한 작은 단위로 나누어야 한다. 이 문자열 단위를 토큰(token)이라고 하고 이렇게 문자열을 토큰으로 나누는 작업을 토큰 생성(tokenizing)이라고 한다. 영문의 경우에는 문장, 단어 등을 토큰으로 사용하거나 정규 표현식을 쓸 수 있다.
- 문자열을 토큰으로 분리하는 함수를 토큰 생성 함수(tokenizer)라고 한다. 토큰 생성 함수는 문자열을 입력받아 토큰 문자열의 리스트를 출력한다.
~~~python
from nltk.tokenize import word_tokenize
word_tokenize(emma_raw[50:100])
~~~
> ['Emma',
 'Woodhouse',
 ',',
 'handsome',
 ',',
 'clever',
 ',',
 'and',
 'rich',
 ',',
 'with',
 'a']
 
### 2-2) from nltk.tag import pos_tag (품사 부착)
 
 - 품사(POS, part-of-speech)는 낱말을 문법적인 기능이나 형태, 뜻에 따라 구분한 것이다. 품사의 구분은 언어마다 그리고 학자마다 다르다. 예를 들어 NLTK에서는 펜 트리뱅크 태그세트(Penn Treebank Tagset)라는 것을 이용한다. 다음은 펜 트리뱅크 태그세트에서 사용하는 품사의 예이다.
     - NNP : 단수 고유명사
     - VB : 동사
     - VBP : 동사 현재형
     - TO : to 전칳사
     - NN : 명사(단수형 혹은 집합형)
     - DT : 관형사
 - pos_tag 명령을 사용하면 단어 토큰에 품사를 부착하여 튜플로 출력한다. 다음 예문에서 refuse, permit이라는 같은 철자의 단어가 각각 동사와 명사로 다르게 품사 부착된 것을 볼 수 있다.
 ~~~python
from nltk.tag import pos_tag
sentence = "Emma refused to permit us to obtain the refuse permit"
tagged_list = pos_tag(word_tokenize(sentence))
tagged_list
~~~
> [('Emma', 'NNP'),
 ('refused', 'VBD'),
 ('to', 'TO'),
 ('permit', 'VB'),
 ('us', 'PRP'),
 ('to', 'TO'),
 ('obtain', 'VB'),
 ('the', 'DT'),
 ('refuse', 'NN'),
 ('permit', 'NN')]
 
 - Scikit-Learn 등에서 자연어 분석을 할 때는 같은 토큰이라도 품사가 다르면 다른 토큰으로 처리해야 하는 경우가 많은데 이 때는 원래의 토큰과 품사를 붙여서 새로운 토큰 이름을 만들어 사용하면 철자가 같고 품사가 다른 단어를 구분할 수 있다.
 
### 2-3) nltk.ne_chunk

- nltk 라이브러리 ne_chunk() 함수를 사용해서 개체명을 인식시킬 수 있다
- 개체명 인식을 사용하면 코퍼스로부터 어떤 단어가 사람, 장소, 조직 등을 의미하는 단어인지를 찾을 수 있습니다.
- 어떤 이름을 의미하는 단어를 보고는 그 단어가 어떤 유형인지를 인식하는 것을 말합니다.
    - "유정이는 2018년에 골드만삭스에 입사했다."
    - 유정 -> 사람 / 2018년 -> 시간 / 골드만삭스 -> 조직
- NLTK에서는 개체명 인식기(NER chunker)를 지원하고 있으므로, 별도 개체명 인식기를 구현할 필요없이 NLTK를 사용해서 개체명 인식을 수행할 수 있습니다.
- ne_chunk는 개체명을 태깅하기 위해서 앞서 품사 태깅(pos_tag)이 수행되어야 합니다. 위의 결과에서 James는 PERSON(사람), Disney는 조직(ORGANIZATION), London은 위치(GPE)라고 정상적으로 개체명 인식이 수행된 것을 볼 수 있습니다.
~~~python
from nltk import word_tokenize, pos_tag, ne_chunk
sentence = "James is working at Disney in London"
sentence=pos_tag(word_tokenize(sentence))
print(sentence) # 토큰화와 품사 태깅을 동시 수행
~~~
> [('James', 'NNP'), ('is', 'VBZ'), ('working', 'VBG'), ('at', 'IN'), ('Disney', 'NNP'), ('in', 'IN'), ('London', 'NNP')]
~~~python
sentence=ne_chunk(sentence)
print(sentence) # 개체명 인식
~~~
> (S
  (PERSON James/NNP)
  is/VBZ
  working/VBG
  at/IN
  (ORGANIZATION Disney/NNP)
  in/IN
  (GPE London/NNP))

### 2-4) from nltk.sentiment.vader import SentimentIntensityAnalyzer

- 문자열을 가져와서 네 가지 범주 각각에 대한 점수 dictionary를 반환합니다.
    - negative
    - neutral
    - positive
    - compound(computed by normalizing the scores above)
    
> a = 'This was a good movie.'
> sid.polarity_scores(a)

> OUTPUT-{'neg': 0.0, 'neu': 0.508, 'pos': 0.492, 'compound': 0.4404}

> a = 'This was the best, most awesome movie EVER MADE!!!'
> sid.polarity_scores(a)

> OUTPUT-{'neg': 0.0, 'neu': 0.425, 'pos': 0.575, 'compound': 0.8877}

### 2-5) from nltk.corpus import stopwords

- 갖고 있는 데이터에서 유의미한 단어 토큰만을 선별하기 위해서는 큰 의미가 없는 단어 토큰을 제거하는 작업이 필요합니다. 
- 여기서 큰 의미가 없다라는 것은 자주 등장하지만 분석을 하는 것에 있어서는 큰 도움이 되지 않는 단어들을 말합니다. 
- 예를 들면, I, my, me, over, 조사, 접미사 같은 단어들은 문장에서는 자주 등장하지만 실제 의미 분석을 하는데는 거의 기여하는 바가 없는 경우가 있습니다. 
- 이러한 단어들을 불용어(stopword)라고 하며, NLTK에서는 위와 같은 100여개 이상의 영어 단어들을 불용어로 패키지 내에서 미리 정의하고 있습니다.
~~~python
from nltk.corpus import stopwords  
stopwords.words('english')[:10]
~~~
> ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your']  


In [5]:
from textstat import flesch_reading_ease

import fasttext

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk import word_tokenize, pos_tag, ne_chunk, tree2conlltags
from nltk.corpus import stopwords

import string
import xgboost as xgb
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn import metrics, model_selection, naive_bayes

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier

nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('vader_lexicon')
nltk.download('maxent_ne_chunker')
nltk.download('words')

eng_stopwords = set(stopwords.words("english"))
symbols_knowns = string.ascii_letters + string.digits + string.punctuation

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marti\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\marti\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\marti\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\marti\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\marti\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\marti\AppData\Roaming\nltk_data...
[nltk_data]   Package words is

- punctuation ; 따옴표, 마침표, 물음표 등등 과 같은 문장부호

In [6]:
def sentiment_nltk(text):
    res = SentimentIntensityAnalyzer().polarity_scores(text)
    return res['compound']

def get_words(text):
    words = nltk.tokenize.word_tokenize(text)
    return [word for word in words if not word in string.punctuation]
    
def count_tokens(text, tokens):
    return sum([w in tokens for w in get_words(text)])

def first_word_len(text):
    if(len(get_words(text))==0):
        return 0
    else:   
        return len(get_words(text)[0])

def last_word_len(text):
    if(len(get_words(text))==0):
        return 0
    else:   
        return len(get_words(text)[-1])

def symbol_id(x):
    symbols=[x for x in symbols_knowns]
      
    if x not in symbols:
        return -1 
    else:
        return np.where(np.array(symbols) == x )[0][0]

In [7]:
## 전체 단어 개수에서 '명사'의 품사를 가진 단어의 비율을 구함.
def fraction_noun(text):
    text_splited = text.split(' ')
    text_splited = [''.join(c for c in s if c not in string.punctuation) for s in text_splited]
    text_splited = [s for s in text_splited if s]
    word_count = text_splited.__len__() # len()과 object.__len__()은 다름. len()은 __len__()을 호출하는 것임.
    if word_count==0:
        return 0
    else:
        pos_list = nltk.pos_tag(text_splited)
        noun_count = len([w for w in pos_list if w[1] in ('NN','NNP','NNPS','NNS')])
    
        return (noun_count/word_count)
    
## 전체 단어 개수에서 '형용사'의 품사를 가진 단어의 비율을 구함.
def fraction_adj(text):
    text_splited = text.split(' ')
    text_splited = [''.join(c for c in s if c not in string.punctuation) for s in text_splited]
    text_splited = [s for s in text_splited if s]
    word_count = text_splited.__len__()
    if word_count==0:
        return 0
    else:
        pos_list = nltk.pos_tag(text_splited)
        adj_count = len([w for w in pos_list if w[1] in ('JJ','JJR','JJS')])
    
        return (adj_count/word_count)  

## 전체 단어 개수에서 '동사'의 품사를 가진 단어의 비율을 구함.
def fraction_verbs(text):
    text_splited = text.split(' ')
    text_splited = [''.join(c for c in s if c not in string.punctuation) for s in text_splited]
    text_splited = [s for s in text_splited if s]
    word_count = text_splited.__len__()
    if word_count==0:
        return 0
    else:
        pos_list = nltk.pos_tag(text_splited)
        verbs_count = len([w for w in pos_list if w[1] in ('VB','VBD','VBG','VBN','VBP','VBZ')])
    
        return (verbs_count/word_count)  

## 생성한 feature

1. 각 문장에 포함된 **'단어의 개수'**
2. 각 문장에 포함된 **'단어의 평균 길이'**
3. 각 문장에 포함된 **'겹치지 않는 단어의 개수'**
4. 각 문장에 포함된 **'문자의 개수'**
5. 각 문장에 포함된 **'stopwards(불용어)의 개수'**
6. 각 문장에 포함된 **'문장부호의 개수'**
7. 각 문자에 포함된 단어 중 **'Upper case로 된 단어의 비율'**
8. 각 문자에 포함된 단어 중 **'title case(upper case + lower case)로 된 단어의 비율'**
9. 각 문장에 포함된 전체 문자의 개수 중 **'','로 구분되어진 chunk에 포함된 문자들의 평균 개수에 대한 비율'**
10. 각 문장에 포함된 전체 문자의 개수 중 **'ascii 문자나 숫자와 같은 symbol의 비율'**
11. 각 문장에 포함된 **'명사의 개수'**
12. 각 문장에 포함된 **'형용사의 개수'**
13. 각 문장에 포함된 **'동사의 개수'**
14. 각 문장의 **'SentimentIntensityAnalyzer의 compound 분석 값'**
15. 각 문장에서 **'단수 주어/주어/목적어 token이 포함된 갯수'**
16. 각 문장에서 **'복수 주어/주어/목적어 token이 포함된 갯수'**
17. 각 문장에 포함된 전체 문자의 개수에 대한 **'첫번째 문자 길이의 비율'**
18. 각 문장에 포함된 전체 문자의 개수에 대한 **'마지막 문자 길이의 비율'**
19. **첫번째 단어의 'symbol id를 구함'**
20. **마지막 단어의 'symbol id를 구함'**
21. **flesch_reading_ease score**를 계산

In [8]:
# 각 문장에 포함된 '단어의 개수'
train['num_words']=train['text'].apply(lambda x:len(get_words(x)))

# 각 문장에 포함된 '단어의 평균 길이'
train['mean_word_len']=train['text'].apply(lambda x:np.mean([len(w) for w in str(x).split()]))

# 각 문장에 포함된 '겹치지 않는 단어의 개수'
train["num_unique_words"] = train["text"].apply(lambda x: len(set(str(x).split())))

# 각 문장에 포함된 '문자의 개수'
train["num_chars"] = train["text"].apply(lambda x: len(str(x)))

# 각 문장에 포함된 'stopwards(불용어)의 개수'
train["num_stopwords"] = train["text"].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))

# 각 문장에 포함된 '문장부호의 개수'
train["num_punctuations"] =train['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))

# 각 문자에 포함된 단어 중 'Upper case로 된 단어의 비율'
train["num_words_upper"] = train["text"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))/train["num_words"]

# 각 문자에 포함된 단어 중 'title case(upper case + lower case)로 된 단어의 비율'
train["num_words_title"] = train["text"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))/train["num_words"]

# 각 문장에 포함된 전체 문자의 개수 중 '','로 구분되어진 chunk에 포함된 문자들의 평균 개수에 대한 비율'
train["chars_between_comma"] = train["text"].apply(lambda x: np.mean([len(chunk) for chunk in str(x).split(",")]))/train["num_chars"]

# 각 문장에 포함된 전체 문자의 개수 중 'ascii 문자나 숫자와 같은 symbol의 비율'
train["symbols_unknowns"]=train["text"].apply(lambda x: np.sum([not w in symbols_knowns for w in str(x)]))/train["num_chars"]

# 각 문장에 포함된 '명사의 개수'
train['noun'] = train["text"].apply(lambda x: fraction_noun(x))

# 각 문장에 포함된 '형용사의 개수'
train['adj'] = train["text"].apply(lambda x: fraction_adj(x))

# 각 문장에 포함된 '동사의 개수'
train['verbs'] = train["text"].apply(lambda x: fraction_verbs(x))

# 각 문장의 'SentimentIntensityAnalyzer의 compound 분석 값'
train["sentiment"]=train["text"].apply(sentiment_nltk)

# 각 문장에서 '단수 주어/주어/목적어 token이 포함된 갯수'
train['single_frac'] = train['text'].apply(lambda x: count_tokens(x, ['is', 'was', 'has', 'he', 'she', 'it', 'her', 'his']))/train["num_words"]

# 각 문장에서 '복수 주어/주어/목적어 token이 포함된 갯수'
train['plural_frac'] = train['text'].apply(lambda x: count_tokens(x, ['are', 'were', 'have', 'we', 'they']))/train["num_words"]

# 각 문장에 포함된 전체 문자의 개수에 대한 '첫번째 문자 길이의 비율'
train['first_word_len']=train['text'].apply(first_word_len)/train["num_chars"]

# 각 문장에 포함된 전체 문자의 개수에 대한 '마지막 문자 길이의 비율'
train['last_word_len']=train['text'].apply(last_word_len)/train["num_chars"]

# 첫번째 단어의 'symbol id를 구함'
train["first_word_id"] = train['text'].apply(lambda x: symbol_id(list(x.strip())[0]))

# 마지막 단어의 'symbol id를 구함'
train["last_word_id"] = train['text'].apply(lambda x: symbol_id(list(x.strip())[-1]))

# flesch_reading_ease score를 계산
train['ease']=train['text'].apply(flesch_reading_ease)


# 동일 feature를 test data에 대해서도 생성
test['num_words']=test['text'].apply(lambda x:len(str(x).split()))
test['mean_word_len']=test['text'].apply(lambda x:np.mean([len(w) for w in str(x).split()]))
test["num_unique_words"] = test["text"].apply(lambda x: len(set(str(x).split())))
test["num_chars"] = test["text"].apply(lambda x: len(str(x)))
test["num_stopwords"] = test["text"].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))
test["num_punctuations"] =test['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]) )
test["num_words_upper"] = test["text"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))/test["num_words"]
test["num_words_title"] = test["text"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))/test["num_words"]
test["chars_between_comma"] = test["text"].apply(lambda x: np.mean([len(chunk) for chunk in str(x).split(",")]))/test["num_chars"]
test["symbols_unknowns"]=test["text"].apply(lambda x: np.sum([not w in symbols_knowns for w in str(x)]))/test["num_chars"]
test['noun'] = test["text"].apply(lambda x: fraction_noun(x))
test['adj'] = test["text"].apply(lambda x: fraction_adj(x))
test['verbs'] = test["text"].apply(lambda x: fraction_verbs(x))
test["sentiment"]=test["text"].apply(sentiment_nltk)
test['single_frac'] = test['text'].apply(lambda x: count_tokens(x, ['is', 'was', 'has', 'he', 'she', 'it', 'her', 'his']))/test["num_words"]
test['plural_frac'] = test['text'].apply(lambda x: count_tokens(x, ['are', 'were', 'have', 'we', 'they']))/test["num_words"]
test['first_word_len']=test['text'].apply(first_word_len)/test["num_chars"]
test['last_word_len']=test['text'].apply(last_word_len)/test["num_chars"]
test["first_word_id"] = test['text'].apply(lambda x: symbol_id(list(x.strip())[0]))
test["last_word_id"] = test['text'].apply(lambda x: symbol_id(list(x.strip())[-1]))
test['ease']=test['text'].apply(flesch_reading_ease)

### 작가별 등장인물의 list를 만들어서 text에 등장하는 인물과의 유사도를 계산하는 듯!

In [11]:
def get_persons(text):
    def bind_names(tagged_words):
        names=list()
        name=list()
        for i,w in enumerate(tagged_words): # 반복문 사용 시 몇 번째 반복문인지 확인 / 인덱스 번호와 컬렉션의 원소를 tuple형태로 반환 
            if("PERSON" in w[2]):
                name.append(w[0])
            else:
                if(len(name)!=0):
                    names.append(" ".join(name))
                name=list()
                
            if(i==len(tagged_words)-1 and len(name)!=0):
                names.append(" ".join(name))
        return names                   

    res_ne_tree = ne_chunk(pos_tag(word_tokenize(text))) # 품사를 태깅한 token을 개체명 인식하여 tree형태로 반환
    res_ne = tree2conlltags(res_ne_tree) # ne_chunk로 인해 tree로 반환된 것을 IOB 형태로 변환하여 줌.
    res_ne_list = [list(x) for x in res_ne] # IOB 형태가 된 token을 list 형태로 변환
    return bind_names(res_ne_list)


text_author_0 = " ".join(list(train['text'][train['author']==0]))
text_author_1 = " ".join(list(train['text'][train['author']==1]))
text_author_2 = " ".join(list(train['text'][train['author']==2]))
text_author_3 = " ".join(list(train['text'][train['author']==3]))
text_author_4 = " ".join(list(train['text'][train['author']==4]))

persons_author_0 = set(get_persons(text_author_0))
persons_author_1 = set(get_persons(text_author_1))
persons_author_2 = set(get_persons(text_author_2))
persons_author_3 = set(get_persons(text_author_3))
persons_author_4 = set(get_persons(text_author_4))

# 자카드 지수(Jaccard index)는 두 집합 사이의 유사도를 측정하는 방법 중 하나이다. 
# 자카드 계수(Jaccard coefficient) 또는 자카드 유사도(Jaccard similarity)라고도 한다. 
# 자카드 지수는 0과 1 사이의 값을 가지며, 두 집합이 동일하면 1의 값을 가지고, 공통의 원소가 하나도 없으면 0의 값을 가진다.
def jaccard(a,b):
    return len(a&b)/len(a|b)

# 작가별 등장 인물의 유사도를 feature로 넣는듯!!

train["persons_0"]=train["text"].apply(lambda x:jaccard(set(get_persons(x)),persons_author_0)) 
train["persons_1"]=train["text"].apply(lambda x:jaccard(set(get_persons(x)),persons_author_1)) 
train["persons_2"]=train["text"].apply(lambda x:jaccard(set(get_persons(x)),persons_author_2)) 
train["persons_3"]=train["text"].apply(lambda x:jaccard(set(get_persons(x)),persons_author_3)) 
train["persons_4"]=train["text"].apply(lambda x:jaccard(set(get_persons(x)),persons_author_4)) 

test["persons_0"]=test["text"].apply(lambda x:jaccard(set(get_persons(x)),persons_author_0)) 
test["persons_1"]=test["text"].apply(lambda x:jaccard(set(get_persons(x)),persons_author_1)) 
test["persons_2"]=test["text"].apply(lambda x:jaccard(set(get_persons(x)),persons_author_2)) 
test["persons_3"]=test["text"].apply(lambda x:jaccard(set(get_persons(x)),persons_author_3)) 
test["persons_4"]=test["text"].apply(lambda x:jaccard(set(get_persons(x)),persons_author_4)) 

KeyboardInterrupt: 

In [ ]:
train['text'].to_csv(data_path + 'sample_file.txt',index=False, header=None, sep="\t")
model_ft = fasttext.train_unsupervised(data_path + 'sample_file.txt', minCount=2, minn=2, maxn=10,dim=300)

def sent2vec(s):
    words = nltk.tokenize.word_tokenize(s)
    #words = [k.stem(w) for w in words]
    #words = [w for w in words if not w in string.digits]
    #words = [w for w in words if w.isalpha()]
    M = []
    for w in words:
        try:
            M.append(model_ft[w])
        except:
            continue
    M = np.array(M)
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
    return v

xtrain_ft = np.array([sent2vec(x) for x in train['text']])
xtest_ft = np.array([sent2vec(x) for x in test['text']])

train_ft=pd.DataFrame(xtrain_ft)
train_ft.columns = ['ft_vector_'+str(i) for i in range(xtrain_ft.shape[1])]

test_ft=pd.DataFrame(xtest_ft)
test_ft.columns = ['ft_vector_'+str(i) for i in range(xtrain_ft.shape[1])]

train = pd.concat([train, train_ft], axis=1)
test = pd.concat([test, test_ft], axis=1)